In [11]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [12]:
df_censo_escolar = pd.read_csv("microdados_ed_basica_2021.csv", sep = ";", encoding = "iso-8859-1")

/tmp/ipykernel_15007/1655325398.py:1: DtypeWarning:

Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.



##### Filtrando os dados, sendo o ultimo filtro especifico para este gráfico.

In [13]:
df_censo_local_dif_in_RS = df_censo_escolar.query(
    "TP_LOCALIZACAO_DIFERENCIADA == 1"
    "| TP_LOCALIZACAO_DIFERENCIADA == 2" 
    "| TP_LOCALIZACAO_DIFERENCIADA == 3"     
)

df_censo_local_dif_in_RS = df_censo_local_dif_in_RS.query("SG_UF == 'RS'")

df_censo_local_dif_in_RS = df_censo_local_dif_in_RS[["TP_LOCALIZACAO_DIFERENCIADA",
    "QT_MAT_BAS","QT_DOC_BAS","TP_LOCALIZACAO","QT_TUR_BAS","NO_ENTIDADE", "NO_MUNICIPIO"]]

##### Tratando os dados, fazendo a alteração do campo TO_LOCALIZACAO de float para STR e excluindo os dados NaN do campo QT_TUR_BAS


In [14]:
# Poderia renomear as colunas, com a função rename, desta forma facilitaria a criação do gráfico, porém optei por fazer os update.

df_censo_local_dif_in_RS["TP_LOCALIZACAO_DIFERENCIADA"] = df_censo_local_dif_in_RS["TP_LOCALIZACAO_DIFERENCIADA"].astype(str)

df_censo_local_dif_in_RS = df_censo_local_dif_in_RS.dropna(subset=["QT_TUR_BAS"])

##### Criando o gráfico

In [15]:
grafic = px.scatter(df_censo_local_dif_in_RS, 
    x= "QT_MAT_BAS", 
    y= "QT_DOC_BAS", 
    facet_col= "TP_LOCALIZACAO",
    size= "QT_TUR_BAS",
    hover_data=["NO_ENTIDADE", "NO_MUNICIPIO"],
    color= "TP_LOCALIZACAO_DIFERENCIADA",
    color_discrete_map= {"1.0": "brown","2.0": "orange", "3.0": "turquoise"},
    size_max= 30,
    template= "ggplot2",
    height= 600,
    width= 1200,
    opacity= .75,
    range_x=[0,500],
    range_y=[0,200]
)

#### Alterações

In [17]:

# Alterando titulo e tamanho da fonte do gráfico
grafic.update_layout(
  title="<b>Censo Escolar 2021: Análise de Localização Diferenciada(RS)<b>",
  font_size= 14
)

#Alterando o nome das legendas:
grafic.for_each_trace(
  lambda t: t.update(
    name= {"1.0": "Área de Assentamento", "2.0": "Terra Indígina", "3.0": "Áreas Remanescentes de Quilombo"}[t.name],
    legendgroup= {"1.0": "Área de Assentamento", "2.0": "Terra Indígina", "3.0": "Áreas Remanescentes de Quilombo"}[t.name])
)

#Aplicando borda nos circulos
grafic.update_traces(marker_line_width = 1, marker_line_color= "black")

# Alterando a posição, fonte e titulo da legenda:
grafic.update_layout(
  legend= dict(
    title= "Localização Diferenciada:",
    orientation= "h",
    font_size= 12,
    yanchor= "top",
    xanchor= "center",
    y= -0.2,
    x= 0.3
  )
)

# Alterando os títulos dos eixos x e y:
grafic.update_layout(
    xaxis_title= "Quantidade de Matículas",
    xaxis2_title= "Quantidade de Matículas",
    yaxis_title= "Quantidade de Docentes",
    font_size= 11
)

# Incluindo a fonte
grafic.add_annotation(
    text= "Fonte: INEP",
    xref= "paper",
    yref= "paper",
    font_color= "darkgrey",
    font_size= 12,
    showarrow= False,
    y= 1.1,
    x= 1,
    
)

# Caso queira que a fonte fique dentro dos gráficos, utilize o campo comentado abaixo:

# grafic.add_annotation(
#  text= "Fonte: INEP",
#  xref= "paper",
#  yref= "paper",
#  font_color= "darkgrey",
#  font_size= 12,
#  showarrow= False,
#  y= 0.99,
#  x= 0.991
# )

# grafic.add_annotation(
#  text= "Fonte: INEP",
#  xref= "paper",
#  yref= "paper",
#  font_color= "darkgrey",
#  font_size= 12,
#  showarrow= False,
#  y= 0.99,
#  x= 0.991
# )

# Renomenado os Gráficos
grafic.layout.annotations[0].text= "Rural"
grafic.layout.annotations[1].text= "Urbano"

##### Gráfico

In [18]:
grafic.show()

#### Exportação

In [ ]:
grafic.write_html("localizacao_diferenciada_rs.html")